# A Simple Neural Network for MNSIT

In [10]:
##Import Packages

import torch
import torch.nn as nn # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F # All functions that don't have any parameters
from torch.utils.data import DataLoader # Gives easier dataset managment and creates mini batches
import torchvision.datasets as datasets # Has standard datasets we can import in a nice and easy way
import torchvision.transforms as transforms # Transformations we can perform on our dataset

In [14]:
batch_size=64
train_dataset=datasets.MNIST(root=r"C:\Users\green\Desktop\Github\Deep-Learning-Using-Pytorch\dataset",train=True,transform=transforms.ToTensor(),download=True)
train_loader=DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_dataset=datasets.MNIST(root=r"C:\Users\green\Desktop\Github\Deep-Learning-Using-Pytorch\dataset",train=False,transform=transforms.ToTensor(),download=True)
test_loader=DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size = 784 # 28x28 = 784, size of MNIST images (grayscale)
num_classes = 10
learning_rate = 0.001
num_epochs = 6

In [27]:
class NeuralNetwork(nn.Module):
    
    def __init__(self, input_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.fc1=nn.Linear(input_size, 50)
        self.fc2=nn.Linear(50, num_classes)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [28]:
model = NeuralNetwork(input_size=input_size,num_classes=num_classes).to(device)
optimizer = optim.Adam(params=model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [29]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy"
            f" {float(num_correct) / float(num_samples) * 100:.2f}"
        )

    model.train()

In [33]:
for epoch in range(num_epochs):
    print(f"Epoch: {epoch}")
    for batch_idx,(data,targets) in enumerate(train_loader):
        data=data.to(device)
        targets = targets.to(device=device)
        
        data=data.reshape(data.shape[0],-1)
        
        results=model(data)
        
        loss=criterion(results,targets)
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
    
    check_accuracy(train_loader, model)
    check_accuracy(test_loader, model)

Epoch: 0
Got 56090 / 60000 with accuracy 93.48
Got 9343 / 10000 with accuracy 93.43
Epoch: 1
Got 57104 / 60000 with accuracy 95.17
Got 9485 / 10000 with accuracy 94.85
Epoch: 2
Got 57706 / 60000 with accuracy 96.18
Got 9548 / 10000 with accuracy 95.48
Epoch: 3
Got 58244 / 60000 with accuracy 97.07
Got 9631 / 10000 with accuracy 96.31
Epoch: 4
Got 58422 / 60000 with accuracy 97.37
Got 9663 / 10000 with accuracy 96.63
Epoch: 5
Got 58701 / 60000 with accuracy 97.84
Got 9686 / 10000 with accuracy 96.86


In [20]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Got 57549 / 60000 with accuracy 95.91
Got 9556 / 10000 with accuracy 95.56
